In [145]:
import pandas as pd

customers = "https://dojo-git.s3.eu-west-3.amazonaws.com/raw/customers.csv"
transactions = "https://dojo-git.s3.eu-west-3.amazonaws.com/raw/transactions.csv"
products = "https://dojo-git.s3.eu-west-3.amazonaws.com/raw/products.csv"

df_cust = pd.read_csv(customers, sep = ',')

df_prod = pd.read_csv(products, sep = ',')

df_tran = pd.read_csv(transactions, sep = ',')

In [146]:
df_cust.head()

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943


In [147]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


In [148]:
df_prod.head()

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0


In [149]:
df_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB


In [150]:
df_tran.head()

,id_prod,date,session_id,client_id
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232


In [151]:
df_tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679532 entries, 0 to 679531
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     679532 non-null  object
 1   date        679532 non-null  object
 2   session_id  679532 non-null  object
 3   client_id   679532 non-null  object
dtypes: object(4)
memory usage: 20.7+ MB


In [152]:
# Jointure des tableaux

df = pd.merge(df_tran,
                  df_cust,
                  how = "left",
                  left_on = "client_id",
                  right_on = "client_id")

In [153]:
df = pd.merge(df,
                  df_prod,
                  how = "left",
                  left_on = "id_prod",
                  right_on = "id_prod")

In [154]:
df.head()

,id_prod,date,session_id,client_id,sex,birth,price,categ
0,0_1518,2022-05-20 13:21:29.043970,s_211425,c_103,f,1986,4.18,0.0
1,1_251,2022-02-02 07:55:19.149409,s_158752,c_8534,m,1988,15.99,1.0
2,0_1277,2022-06-18 15:44:33.155329,s_225667,c_6714,f,1968,7.99,0.0
3,2_209,2021-06-24 04:19:29.835891,s_52962,c_6941,m,2000,69.99,2.0
4,0_1509,2023-01-11 08:22:08.194479,s_325227,c_4232,m,1980,4.99,0.0


In [155]:
# Suppresion des valeurs manquantes

df = df.dropna()

In [156]:
df.describe()

,birth,price,categ
count,679311.000000,679311.000000,679311.000000
mean,1977.817976,17.449340,0.441823
std,13.579191,18.329034,0.595002
min,1929.000000,-1.000000,0.000000
25%,1970.000000,8.820000,0.000000
50%,1980.000000,13.990000,0.000000
75%,1987.000000,18.990000,1.000000
max,2004.000000,300.000000,2.000000


In [157]:
# Correction des incohérences

# Suppression de produits qui coutent 0 ou moins 

df = df[df['price'] > 0]

# Arrondir les prix

df['price'] = df['price'].round(2)

In [158]:
# Standardisation des formats date and rounding secondes

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = df['date'].dt.floor('S')

df.head()

,id_prod,date,session_id,client_id,sex,birth,price,categ
0,0_1518,2022-05-20 13:21:29,s_211425,c_103,f,1986,4.18,0.0
1,1_251,2022-02-02 07:55:19,s_158752,c_8534,m,1988,15.99,1.0
2,0_1277,2022-06-18 15:44:33,s_225667,c_6714,f,1968,7.99,0.0
3,2_209,2021-06-24 04:19:29,s_52962,c_6941,m,2000,69.99,2.0
4,0_1509,2023-01-11 08:22:08,s_325227,c_4232,m,1980,4.99,0.0


In [159]:
# Transformer id_prod, session_id et client_id en int, supression des valuers str

df['id_prod'] = df['id_prod'].str[2:].astype(int)
df['session_id'] = df['session_id'].str[2:].astype(int)
df['client_id'] = df['client_id'].str[2:].astype(int)


In [160]:
# Transformer category en int 

df['categ'] = df['categ'].astype(int)

In [161]:
# Factorize le sexe

df['sex'], uniques = pd.factorize(df['sex'])

In [162]:
df.head()

,id_prod,date,session_id,client_id,sex,birth,price,categ
0,1518,2022-05-20 13:21:29,211425,103,0,1986,4.18,0
1,251,2022-02-02 07:55:19,158752,8534,1,1988,15.99,1
2,1277,2022-06-18 15:44:33,225667,6714,0,1968,7.99,0
3,209,2021-06-24 04:19:29,52962,6941,1,2000,69.99,2
4,1509,2023-01-11 08:22:08,325227,4232,1,1980,4.99,0


In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 679111 entries, 0 to 679531
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     679111 non-null  int32         
 1   date        679111 non-null  datetime64[ns]
 2   session_id  679111 non-null  int32         
 3   client_id   679111 non-null  int32         
 4   sex         679111 non-null  int64         
 5   birth       679111 non-null  int64         
 6   price       679111 non-null  float64       
 7   categ       679111 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(4), int64(2)
memory usage: 36.3 MB


In [164]:
df.to_csv('cleaned.csv', index=False) 